# Projeto de Aprendizagem Automática II

## Procura de Exoplanetas no Espaço através da Emissão de Luz de Estrelas

### Importação de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

### Carregamento dos Dados

O primeiro passo é o carregamento dos dados, de modo a treinar e, consequentemente, testar o modelo gerado.

In [2]:
treino = pd.read_csv("../../../../Dados/dados_treino.csv")
teste = pd.read_csv("../../../../Dados/dados_teste.csv")

Após o carregamento dos mesmos, é necessário dividir os registos em *labels* e *features*. Note-se que esta divisão deve ser efetuada tanto no conjunto de treino, como no de teste.

In [3]:
X_train = treino.loc[:, treino.columns != 'LABEL'].values
y_train = treino.LABEL.values

X_test = teste.loc[:, teste.columns != 'LABEL'].values
y_test = teste.LABEL.values

Tendo em conta que os melhores resultados do modelo de *SVM* foram obtidos com a aplicação de *PCA* aos dados, esta é a abordagem a seguir, de modo a tentar obter os melhores resultados possíveis para este tipo de classificador.

In [4]:
from sklearn.decomposition import PCA
pca = PCA(0.90, True, False)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
print("the dimension of decomposed data is ", X_train.shape[1])

the dimension of decomposed data is  125


### Modelo

De modo a otimizar os parâmetros do modelo, estes devem ser definidos na forma de dicionário. Tendo em conta o custo computacional deste tipo de operação de procura dentro de várias combinações, apenas foram otimizados os parâmetros *C*, *gamma* e *kernel*, com os valores que se podem ver em seguida.

In [12]:
grid_param = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.05, 0.01, 0.005, 0.001],'kernel': ['rbf', 'poly', 'sigmoid', 'linear']}

Foi tirado proveito do método *GridSearchCV*, permitindo uma pesquisa em grelha dentro das combinações de parâmetros pretendidas, em que exista uma validação em cruz. Neste caso, optou-se por uma validação cruzada de 10 subconjuntos, sendo a métrica de *accuracy* a métrica a observar.

In [13]:
from sklearn.model_selection import GridSearchCV
gd_sr = GridSearchCV(estimator=SVC(),
                     param_grid=grid_param,
                     scoring='accuracy',
                     cv=10,
                     n_jobs=-1,
                     verbose=2)

Tendo o modelo definido e pronto a efetuar a procura, apenas é necessário aplicar o método *fit* com o conjunto de dados de treino. Não faria sentido aplicar a validação cruzada aos dados de teste juntamente com os de treino, já que o modelo ficaria demasiado ajustado aos dados, perdendo a capacidade de classificar corretamente novos casos. 

In [14]:
gd_sr.fit(X_train, y_train)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 14.2min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'gamma': [1, 0.1, 0.05, 0.01, 0.005, 0.001],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=2)

Como se pode ver em seguida, os melhores parâmetros testados são *C* com valor 0.001, *gamma* a 1 e o *kernel* polinomial. 

In [15]:
best_parameters = gd_sr.best_params_
print(best_parameters)

{'C': 0.001, 'gamma': 1, 'kernel': 'poly'}


O melhor resultado obtido para a pesquisa em grelha, com os parâmetros acima mencionados, permitiu uma *accuracy* de sensivelmente 99.3%.

In [16]:
best_result = gd_sr.best_score_
print(best_result)

0.9931492927942978


### Teste

Conhecidos os melhores parâmetros para este modelo, é necessário ver de que forma se comporta na classificação do conjunto de teste. 

In [17]:
preds = gd_sr.predict(X_test)

Observando as métricas obtidas pela predição dos registos de teste, é notório que a precisão da classe minoritária baixou quando em comparação com o modelo original (o qual tinha valor de 60%), mantendo-se o *recall*. 

In [18]:
print(metrics.classification_report(y_test, preds))
print("accuracy:", metrics.accuracy_score(y_test, preds))

              precision    recall  f1-score   support

           1       1.00      0.99      1.00       565
           2       0.50      0.60      0.55         5

    accuracy                           0.99       570
   macro avg       0.75      0.80      0.77       570
weighted avg       0.99      0.99      0.99       570

accuracy: 0.9912280701754386


Observando a matriz de confusão e, mais uma vez, comparando com o modelo original, foram classificados 3 dos 5 sistemas da classe minoritária corretamente, sendo que agora, foi classificado de forma errada mais um sistema como possuindo exoplanetas, perfazendo um total de 3 sistemas sem exoplanetas incorretamente classificados.

In [19]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, preds))

[[562   3]
 [  2   3]]


Em jeito de conclusão sobre este classificador, apesar da tentativa de otimização, não foi possível melhorar os resultados previamente obtidos. Assim, este tipo de classificador dificilmente permitirá obter melhores resultados do que os obtidos para os dados em estudo.